# <center>CI9-STR-08: Structural Dynamics</center>

## <center>Damped free vibrations of a Single Degre-of-Freedom system</center>

<center>**Dr C Málaga-Chuquitaype ([email](mailto:c.malaga@imperial.ac.uk), [webpage](http://www.imperial.ac.uk/emerging-structural-technologies))**</center>

***

The Equation of Motion of the System is:


$m \ddot{u}(t) + c \dot{u}(t) + ku(t) = 0$

Or in mass normalized terms:

$\ddot{u}(t) + 2 \xi \omega_n \dot{u}(t) + \omega_n^2 u(t)$

And its solution is:

$u(t) = e^{-\xi \omega_nt} \left( A_1 \cos{\omega_dt} + A_2 \sin{\omega_dt} \right)$

$A_1 = u_0$

$A_2 = \dfrac{v_0 + \xi \omega_n u_0}{\omega_d}$

<font color='red'> PLEASE NOTE  that the interactive portions of this notebook will run slowly or may not run in the online notebook viewer. You may need to run it locally. If you wish to install Jupyter in your own computer you can find detailed instructions here: http://jupyter.org/install</font>

In [1]:
# Import the NumPy functions with namespace (nickname) np
import numpy as np

In [2]:
# Se-up to display plots inline
%matplotlib inline

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [3]:
# Import the plotting functions
import matplotlib.pyplot as plt

In [4]:
# Import the IPython widgets
from ipywidgets.widgets import interact, interactive, HBox, Label
from ipywidgets import widgets
from IPython.display import display

In [5]:
# Create a time array/vector of 0-25s with 5001 samples
t = np.linspace(0,25, 5001)

In [6]:
def plot_SDOF(u0=0.04, v0=0.4, xi=0.05):
#define the period of the SDOF (these will not change) [in seconds]
    T=1.0
#Based on this we get its natural circular frequency    [in rad/s] 
    w=2*np.pi/T
#And the damped circular frequency    [in rad/s] 
    wd=w*np.sqrt(1-xi**2)
    
#the response of the system from the initial conditions
    A1=u0
    A2=(v0+xi*w*u0)/wd

    response = np.exp(-xi*w*t)*(A1*np.cos(wd*t) + A2*np.sin(wd*t)) 

#we use the FFT algorithm

    freq,amp = mod_FFT(response,t,False)

# Set figure parameters
    fig, (ax1,ax2) = plt.subplots(1,2, figsize = (12,4))

    plt.subplots_adjust(bottom=0.12,left=0.17,top=0.96,right=0.96)
    plt.setp(ax1.get_ymajorticklabels(),family='arial',fontsize=14)
    plt.setp(ax1.get_xmajorticklabels(),family='arial',fontsize=14)
    
    ax1.spines['right'].set_color('none')
    ax1.spines['top'].set_color('none')
    ax1.xaxis.set_ticks_position('bottom')
    ax1.yaxis.set_ticks_position('left')
    ax1.grid(True,linestyle=':',color='0.75')
    ax1.set_axisbelow(True)
    
    ax2.spines['right'].set_color('none')
    ax2.spines['top'].set_color('none')
    ax2.xaxis.set_ticks_position('bottom')
    ax2.yaxis.set_ticks_position('left')
    ax2.grid(True,linestyle=':',color='0.75')
    ax2.set_axisbelow(True)

    #Response
    ax1.set_xlabel('Time [s]', family='arial', fontsize=14,weight='bold',labelpad=5)
    ax1.set_ylabel('Response Amplitude [m]', family='arial', fontsize=14,weight='bold',labelpad=10)
    
    ax1.plot(t, response, linewidth=2,linestyle='-',label='Response')
    ax1.set_xlim(0,5)
    
    #FFT
    ax2.set_xlabel('Frequency [Hz]', fontsize=14,weight='bold',labelpad=5)
    ax2.set_ylabel('Fourier Amplitude', fontsize=14,weight='bold',labelpad=5)
    
    ax2.plot(freq, amp, linewidth=2, linestyle='-', label = 'FFT')
    ax2.set_xlim(-5,20)
    
    #Adjustments
    plt.tight_layout(pad=0.5, w_pad=5.0)    

The function below implements the FFT algorithm. Although not strictly necesary, it also implements additional steps such as zero-centering, selecting the Real part, etc. 

In [7]:
def mod_FFT(data,time,plotflag):
    #Inputs:
    #time = time array corresponding to the data
    #data = the response data arrray
    #pltoflag = plots if nonzero
    #Outputs:
    #fft_freq = array of the frequencies used in the FFT
    #fft_amp = array of the amplitude of the FFT at each frequency
    
    from scipy.fftpack import fft
    
    #any DC offset
    offset = np.mean(data)
    
    #get sampling time
    sample_time = time[1]-time[0]
    
    #get length
    n = len(data)
    
    #calculate the FFT removing the offset and using a Hanning Window
    fft_amp = fft((data-offset)*np.hanning(len(data)))
    
    #define the frequency range of the output
    fft_freq = np.linspace(0.0, 1.0 / (2.0*sample_time), int(np.ceil(n/2)))
    
    #get the fft part of interest
    fft_amp = 2.0/n*np.abs(fft_amp[0:int(np.ceil(n/2))])
    
    #if plotflag is nonzero (True), plot the FFT before returning the mag and phase
    if plotflag:
        fig=plt.figure(figsize=(6,4))
        ax=plt.gca()
        plt.subplots_adjust(bottom=0.17,left=0.17,top=0.96,right=0.96)
        plt.setp(ax.get_ymajorticklabels(), fontsize=14)
        plt.setp(ax.get_xmajorticklabels(), fontsize=14)
        ax.spines['right'].set_color('none')
        ax.spines['top'].set_color('none')
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')
        ax.grid(True, linestyle=':', color='0.75')
        ax.set_axisbelow(True)
        
        plt.xlabel('Frequency [Hz]', fontsize=14,labelpad=8)
        plt.ylabel('Fourier Amplitude', fontsize=14,labelpad=10)
        
        plt.plot(fft_freq,fft_amp,linewidth=2,linestyle='-')
        
        #Adjustements
        plt.tight_layout(pad=0.5)
        plt.show()
        
    return fft_freq, fft_amp  

Now, let's call the interact function, set-up the ranges for each parameter and define the sliders.

In [8]:
#Call the slider interaction, the phi angles are defined as a function of pi/2
interact(plot_SDOF, u0 = (-0.1,0.1,0.05),
         v0 = (-1,1,0.05),
         xi = (0,0.5,0.05))
                    

interactive(children=(FloatSlider(value=0.04, description='u0', max=0.1, min=-0.1, step=0.05), FloatSlider(val…

<function __main__.plot_SDOF>